In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import os

In [3]:
import uuid

In [4]:
import pickle
import pandas as pd

In [5]:
import mlflow

In [6]:
import xgboost as xgb

In [7]:
from google.cloud import storage

In [8]:
def download_directory(bucket_name, directory_path, destination_path):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    blobs = bucket.list_blobs(prefix=directory_path)

    for blob in blobs:
        # Extract the relative path from the blob's name
        relative_path = os.path.relpath(blob.name, directory_path)

        # Create the destination directory if it doesn't exist
        dest_directory = os.path.join(destination_path, os.path.dirname(relative_path))
        os.makedirs(dest_directory, exist_ok=True)

        # Download the blob to the destination directory
        destination_file = os.path.join(dest_directory, os.path.basename(relative_path))
        blob.download_to_filename(destination_file)

    print("Directory downloaded successfully.")



In [9]:
bucket_name='mlops-zoomcamp'

In [10]:
directory_path='2/45e0e8d445544eef87226b6cbbf1921f/artifacts/models_mlflow/'

In [11]:
destination_path="/home/konradballegro/notebooks/inputs/45e0e8d445544eef87226b6cbbf1921f"

In [ ]:
download_directory(bucket_name, directory_path, destination_path)

In [ ]:
# def download_model(bucket_name, artifact_path, destination_path):
#     storage_client = storage.Client()
#     bucket = storage_client.bucket(bucket_name)
#     blob = bucket.blob(artifact_path)
#     blob.download_to_filename(destination_path)

In [ ]:
# download_model(
#         bucket_name='mlops-zoomcamp',
#         artifact_path='data/green_tripdata_2021-02.parquet',
#         destination_path="/home/konradballegro/notebooks/inputs/model.xgb"
#         )

In [ ]:
with open('/home/konradballegro/notebooks/inputs/model.bin', 'rb') as f_in:
    dv, model_old = pickle.load(f_in)

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

In [ ]:
def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [ ]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [ ]:
df.info()

In [ ]:
RUN_ID='8e35278e4321423d8029f87ec106243f'

In [ ]:
input_file='https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet'

In [ ]:
output_file='/home/konradballegro/notebooks/outputs/prediction.parquet'

In [ ]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

In [ ]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['ride_id'] = generate_uuids(len(df))
    
    categorical = ['PULocationID', 'DOLocationID']
    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)

#     return df
    return X_val

In [ ]:
def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [ ]:
def load_model(run_id):
#     logged_model = f's3://mlflow-models-alexey/1/{RUN_ID}/artifacts/model'
#     logged_model=f'gs://mlops-zoomcamp/2/{RUN_ID}/artifacts/models_mlflow/MLmodel'
    model=mlflow.pyfunc.load_model('/home/konradballegro/notebooks/inputs/45e0e8d445544eef87226b6cbbf1921f/')
    return model

In [ ]:
def apply_model(input_file, run_id, output_file):

    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    
    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['tpep_pickup_datetime'] = df['tpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id
    
    df_result.to_parquet(output_file, index=False)

In [ ]:
apply_model(input_file=input_file, run_id=RUN_ID, output_file=output_file)

In [ ]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)